使用Numpy实现神经网络
===

# 1.导入必要的库与包

In [3]:
import numpy as np
from JLib.pyn.layers import Conv2D, Relu, MaxPooling, FullyConnect, Softmax
from jjzhk.progressbar import ProgressBar
import time
import struct
from glob import glob

# 2.加载MNIST数据集

In [4]:
def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    images_path = glob('%s/%s*3-ubyte' % (path, kind))[0]
    labels_path = glob('%s/%s*1-ubyte' % (path, kind))[0]

    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II',
                                 lbpath.read(8))
        labels = np.fromfile(lbpath,
                             dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack('>IIII',
                                               imgpath.read(16))
        images = np.fromfile(imgpath,
                             dtype=np.uint8).reshape(len(labels), 784)

    return images, labels

In [5]:
images, labels = load_mnist('/Users/JJZHK/data/input/MNIST/raw')
test_images, test_labels = load_mnist('/Users/JJZHK/data/input/MNIST/raw', 't10k')

# 3.定义网络

In [6]:
batch_size = 64

conv1 = Conv2D([batch_size, 28, 28, 1], 12, 5, 1)
relu1 = Relu(conv1.output_shape)
pool1 = MaxPooling(relu1.output_shape)
conv2 = Conv2D(pool1.output_shape, 24, 3, 1)
relu2 = Relu(conv2.output_shape)
pool2 = MaxPooling(relu2.output_shape)
fc = FullyConnect(pool2.output_shape, 10)
sf = Softmax(fc.output_shape)

# 4.开始训练

In [ ]:
bar = ProgressBar(20, int(images.shape[0] / batch_size), "loss:%f, acc:%f")
for epoch in range(1, 21):
    learning_rate = 1e-4

    batch_loss = 0
    batch_acc = 0
    val_acc = 0
    val_loss = 0

    # train
    train_acc = 0
    train_loss = 0
    for i in range(int(images.shape[0] / batch_size)):
        img = images[i * batch_size:(i + 1) * batch_size].reshape([batch_size, 28, 28, 1])
        label = labels[i * batch_size:(i + 1) * batch_size]

        conv1_out = conv1.forward(img)
        relu1_out = relu1.forward(conv1_out)
        pool1_out = pool1.forward(relu1_out)
        conv2_out = conv2.forward(pool1_out)
        relu2_out = relu2.forward(conv2_out)
        pool2_out = pool2.forward(relu2_out)
        fc_out = fc.forward(pool2_out)
        batch_loss += sf.cal_loss(fc_out, np.array(label))
        train_loss += sf.cal_loss(fc_out, np.array(label))

        for j in range(batch_size):
            if np.argmax(sf.softmax[j]) == label[j]:
                batch_acc += 1
                train_acc += 1

        sf.gradient()
        conv1.gradient(relu1.gradient(pool1.gradient(
            conv2.gradient(relu2.gradient(pool2.gradient(
                fc.gradient(sf.eta)))))))

        if i % 1 == 0:
            fc.backward(alpha=learning_rate, weight_decay=0.0004)
            conv2.backward(alpha=learning_rate, weight_decay=0.0004)
            conv1.backward(alpha=learning_rate, weight_decay=0.0004)

        bar.show(epoch, batch_loss / (batch_size * (i + 1)), batch_acc / (batch_size * (i + 1)))


Epoch:1 / 20 [***************                                                                                         ] 141 / 937 ,loss:0.809692, acc:0.759198,remain=0:22:01